# Multi-Selection Edge Group - Fan-Out Parallel Branching

## Overview
This notebook demonstrates **multi-selection edge groups** for one-to-many parallel routing. You'll learn how to:
- Use `add_multi_selection_edge_group()` to fan out from one executor to multiple targets
- Implement selection functions that choose **multiple downstream branches** based on analysis
- Share state across parallel branches for coordinated processing
- Merge results from multiple branches back into workflow state
- Apply conditional persistence logic based on data characteristics

## What This Sample Does
The workflow implements an **advanced email triage system** with parallel processing:
1. **store_email** - Persists email in shared state
2. **email_analysis_agent** - Classifies email (NotSpam/Spam/Uncertain)
3. **to_analysis_result** - Enriches with metadata (email length)
4. **Multi-Selection Routing:**
   - **Spam** → `handle_spam` only
   - **NotSpam (short)** → `submit_to_email_assistant` only
   - **NotSpam (long)** → `submit_to_email_assistant` **AND** `summarize_email` (parallel!)
   - **Uncertain** → `handle_uncertain` only
5. **Conditional Persistence:**
   - Short emails → Direct to database
   - Long emails → Wait for summary, then to database

**Key Feature:** Long NotSpam emails trigger **two parallel branches** simultaneously!

## Key Differences from Switch-Case
- **Switch-Case:** One-of-N routing (exactly one path)
- **Multi-Selection:** One-to-Many routing (multiple paths)
- **Use Case:** When a message needs processing by **multiple** executors in parallel

## Step 1: Import Required Libraries

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

"""Step 06b — Multi-Selection Edge Group sample."""

from dotenv import load_dotenv
import asyncio
import os
from dataclasses import dataclass
from typing import Literal
from uuid import uuid4

from agent_framework import (

    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowEvent,
    WorkflowOutputEvent,
    executor,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel
from typing_extensions import Never

load_dotenv('../../.env')

## Step 2: Define Constants, Models, and Custom Events

**Constants:**
- `EMAIL_STATE_PREFIX` - Namespace for email storage
- `CURRENT_EMAIL_ID_KEY` - Pointer to current email
- `LONG_EMAIL_THRESHOLD` - Length threshold for parallel summarization (100 chars)

**Pydantic Models:**
- `AnalysisResultAgent` - Structured spam detection output
- `EmailResponse` - Drafted reply from assistant
- `EmailSummaryModel` - Summary from summarization agent

**Dataclasses:**
- `Email` - In-memory email record
- `AnalysisResult` - Enriched analysis with length and summary

**Custom Events:**
- `DatabaseEvent` - Simulates database operations

In [ ]:
EMAIL_STATE_PREFIX = "email:"
CURRENT_EMAIL_ID_KEY = "current_email_id"
LONG_EMAIL_THRESHOLD = 100


class AnalysisResultAgent(BaseModel):
    spam_decision: Literal["NotSpam", "Spam", "Uncertain"]
    reason: str


class EmailResponse(BaseModel):
    response: str


class EmailSummaryModel(BaseModel):
    summary: str


@dataclass
class Email:
    email_id: str
    email_content: str


@dataclass
class AnalysisResult:
    spam_decision: str
    reason: str
    email_length: int  # Used for routing decision
    email_summary: str  # Populated by summarize branch
    email_id: str


class DatabaseEvent(WorkflowEvent): ...

## Step 3: Define Executor Functions

**store_email:**
- Entry point, persists email with UUID
- Forwards to analysis agent

**to_analysis_result:**
- Transforms agent output to `AnalysisResult`
- **Enriches** with `email_length` from shared state
- This length drives multi-selection routing!

**submit_to_email_assistant:**
- NotSpam path executor
- Retrieves email, sends to drafting agent

**finalize_and_send:**
- Terminal executor for NotSpam path
- Parses and yields drafted reply

**summarize_email:**
- **Parallel branch** for long NotSpam emails
- Sends to summarization agent

**merge_summary:**
- **Merges summary** back into `AnalysisResult`
- Allows database executor to access summary

**handle_spam / handle_uncertain:**
- Terminal executors for Spam/Uncertain paths

**database_access:**
- Simulates persistence (with delay)
- Emits custom `DatabaseEvent`

In [ ]:
@executor(id="store_email")
async def store_email(email_text: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    new_email = Email(email_id=str(uuid4()), email_content=email_text)
    await ctx.set_shared_state(f"{EMAIL_STATE_PREFIX}{new_email.email_id}", new_email)
    await ctx.set_shared_state(CURRENT_EMAIL_ID_KEY, new_email.email_id)

    await ctx.send_message(
        AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=new_email.email_content)], should_respond=True)
    )


@executor(id="to_analysis_result")
async def to_analysis_result(response: AgentExecutorResponse, ctx: WorkflowContext[AnalysisResult]) -> None:
    parsed = AnalysisResultAgent.model_validate_json(response.agent_run_response.text)
    email_id: str = await ctx.get_shared_state(CURRENT_EMAIL_ID_KEY)
    email: Email = await ctx.get_shared_state(f"{EMAIL_STATE_PREFIX}{email_id}")
    await ctx.send_message(
        AnalysisResult(
            spam_decision=parsed.spam_decision,
            reason=parsed.reason,
            email_length=len(email.email_content),  # Key for routing!
            email_summary="",
            email_id=email_id,
        )
    )


@executor(id="submit_to_email_assistant")
async def submit_to_email_assistant(analysis: AnalysisResult, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    if analysis.spam_decision != "NotSpam":
        raise RuntimeError("This executor should only handle NotSpam messages.")

    email: Email = await ctx.get_shared_state(f"{EMAIL_STATE_PREFIX}{analysis.email_id}")
    await ctx.send_message(
        AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=email.email_content)], should_respond=True)
    )


@executor(id="finalize_and_send")
async def finalize_and_send(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    parsed = EmailResponse.model_validate_json(response.agent_run_response.text)
    await ctx.yield_output(f"Email sent: {parsed.response}")


@executor(id="summarize_email")
async def summarize_email(analysis: AnalysisResult, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    # Only called for long NotSpam emails by selection_func
    email: Email = await ctx.get_shared_state(f"{EMAIL_STATE_PREFIX}{analysis.email_id}")
    await ctx.send_message(
        AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=email.email_content)], should_respond=True)
    )


@executor(id="merge_summary")
async def merge_summary(response: AgentExecutorResponse, ctx: WorkflowContext[AnalysisResult]) -> None:
    summary = EmailSummaryModel.model_validate_json(response.agent_run_response.text)
    email_id: str = await ctx.get_shared_state(CURRENT_EMAIL_ID_KEY)
    email: Email = await ctx.get_shared_state(f"{EMAIL_STATE_PREFIX}{email_id}")
    # Build an AnalysisResult mirroring to_analysis_result but with summary
    await ctx.send_message(
        AnalysisResult(
            spam_decision="NotSpam",
            reason="",
            email_length=len(email.email_content),
            email_summary=summary.summary,
            email_id=email_id,
        )
    )


@executor(id="handle_spam")
async def handle_spam(analysis: AnalysisResult, ctx: WorkflowContext[Never, str]) -> None:
    if analysis.spam_decision == "Spam":
        await ctx.yield_output(f"Email marked as spam: {analysis.reason}")
    else:
        raise RuntimeError("This executor should only handle Spam messages.")


@executor(id="handle_uncertain")
async def handle_uncertain(analysis: AnalysisResult, ctx: WorkflowContext[Never, str]) -> None:
    if analysis.spam_decision == "Uncertain":
        email: Email | None = await ctx.get_shared_state(f"{EMAIL_STATE_PREFIX}{analysis.email_id}")
        await ctx.yield_output(
            f"Email marked as uncertain: {analysis.reason}. Email content: {getattr(email, 'email_content', '')}"
        )
    else:
        raise RuntimeError("This executor should only handle Uncertain messages.")


@executor(id="database_access")
async def database_access(analysis: AnalysisResult, ctx: WorkflowContext[Never, str]) -> None:
    # Simulate DB writes for email and analysis (and summary if present)
    await asyncio.sleep(0.05)
    await ctx.add_event(DatabaseEvent(f"Email {analysis.email_id} saved to database."))

## Step 4: Define the Selection Function

This is the **key function** for multi-selection routing!

**Function Signature:**
```python
def select_targets(analysis: AnalysisResult, target_ids: list[str]) -> list[str]
```

**Parameters:**
- `analysis` - The message from upstream (`to_analysis_result`)
- `target_ids` - List of possible downstream executor IDs (in order)

**Returns:**
- List of executor IDs to invoke (can be empty, one, or multiple!)

**Logic:**
```python
target_ids order: [handle_spam, submit_to_email_assistant, summarize_email, handle_uncertain]

if Spam:     return [handle_spam]
if NotSpam:
    if short: return [submit_to_email_assistant]
    if long:  return [submit_to_email_assistant, summarize_email]  # PARALLEL!
else (Uncertain): return [handle_uncertain]
```

**Key Insight:** Long NotSpam emails return **TWO** targets → parallel execution!

In [ ]:
def select_targets(analysis: AnalysisResult, target_ids: list[str]) -> list[str]:
    """Selection function that chooses one or more downstream branches.
    
    Args:
        analysis: The AnalysisResult from to_analysis_result
        target_ids: [handle_spam, submit_to_email_assistant, summarize_email, handle_uncertain]
    
    Returns:
        List of target executor IDs to invoke (can be multiple for parallel execution)
    """
    handle_spam_id, submit_to_email_assistant_id, summarize_email_id, handle_uncertain_id = target_ids
    
    if analysis.spam_decision == "Spam":
        return [handle_spam_id]
    
    if analysis.spam_decision == "NotSpam":
        targets = [submit_to_email_assistant_id]
        if analysis.email_length > LONG_EMAIL_THRESHOLD:
            # Long email: Add summarization as PARALLEL branch
            targets.append(summarize_email_id)
        return targets
    
    return [handle_uncertain_id]

## Step 5: Build the Multi-Selection Workflow

**Workflow Structure:**
```
store_email → email_analysis_agent → to_analysis_result
                                           │
              ┌────────────────────────────┼─────────────────────────┐
              │                            │                         │
           Spam                      NotSpam                   Uncertain
              │                            │                         │
              ▼                     ┌──────┴──────┐                  ▼
        handle_spam          short │             │ long      handle_uncertain
                                   ▼             ▼
                    submit_to_assistant   summarize_email
                                   │             │
                                   ▼             ▼
                         email_assistant   summary_agent
                                   │             │
                                   ▼             ▼
                            finalize_send   merge_summary
                                   │             │
                                   └──────┬──────┘
                                          ▼
                                   database_access
```

**Key Points:**
- `add_multi_selection_edge_group()` creates the fan-out
- `selection_func=select_targets` determines which branches activate
- Long NotSpam emails trigger **both** assistant and summary branches
- Conditional edges route to database based on email length

In [ ]:
async def run_workflow():
    """Main function to build and run the multi-selection workflow."""
    # Agents
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )

    email_analysis_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "You are a spam detection assistant that identifies spam emails. "
                "Always return JSON with fields 'spam_decision' (one of NotSpam, Spam, Uncertain) "
                "and 'reason' (string)."
            ),
            response_format=AnalysisResultAgent,
        ),
        id="email_analysis_agent",
    )

    email_assistant_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "You are an email assistant that helps users draft responses to emails with professionalism."
            ),
            response_format=EmailResponse,
        ),
        id="email_assistant_agent",
    )

    email_summary_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=("You are an assistant that helps users summarize emails."),
            response_format=EmailSummaryModel,
        ),
        id="email_summary_agent",
    )

    # Build the workflow
    workflow = (
        WorkflowBuilder()
        .set_start_executor(store_email)
        .add_edge(store_email, email_analysis_agent)
        .add_edge(email_analysis_agent, to_analysis_result)
        # Multi-selection fan-out: choose one or more targets based on analysis
        .add_multi_selection_edge_group(
            to_analysis_result,
            [handle_spam, submit_to_email_assistant, summarize_email, handle_uncertain],
            selection_func=select_targets,
        )
        # NotSpam drafting path
        .add_edge(submit_to_email_assistant, email_assistant_agent)
        .add_edge(email_assistant_agent, finalize_and_send)
        # NotSpam summarization path (parallel to drafting)
        .add_edge(summarize_email, email_summary_agent)
        .add_edge(email_summary_agent, merge_summary)
        # Conditional database persistence
        # Short emails: direct from to_analysis_result
        .add_edge(to_analysis_result, database_access, condition=lambda r: r.email_length <= LONG_EMAIL_THRESHOLD)
        # Long emails: after summary is merged
        .add_edge(merge_summary, database_access)
        .build()
    )

    # Read an email sample
    resources_path = os.path.join(
        os.path.dirname(os.path.dirname(os.path.realpath(__file__))),
        "resources",
        "email.txt",
    )
    if os.path.exists(resources_path):
        with open(resources_path, encoding="utf-8") as f:  # noqa: ASYNC230
            email = f.read()
    else:
        print("Unable to find resource file, using default text.")
        email = "Hello team, here are the updates for this week..."

    print("=== Email Content ===")
    print(email[:150] + "..." if len(email) > 150 else email)
    print(f"\nEmail length: {len(email)} chars")
    print(f"Threshold: {LONG_EMAIL_THRESHOLD} chars")
    print(f"Will trigger summarization: {len(email) > LONG_EMAIL_THRESHOLD}")
    print("\n=== Processing ===")

    # Print outputs and database events from streaming
    async for event in workflow.run_stream(email):
        if isinstance(event, DatabaseEvent):
            print(f"📊 Database: {event.data}")
        elif isinstance(event, WorkflowOutputEvent):
            print(f"\n=== Workflow Output ===")
            print(event.data)

## Step 6: Execute the Workflow
**Note:** Requires Azure OpenAI credentials. Authenticate with `az login`.

In [ ]:
# Uncomment to run (requires Azure OpenAI access)
# await run_workflow()

## Expected Output (NotSpam - Long Email)
```
=== Email Content ===
Subject: Team Meeting Follow-up - Action Items

Hi Sarah,

I wanted to follow up on our team meeting this morning...

Email length: 285 chars
Threshold: 100 chars
Will trigger summarization: True

=== Processing ===

📊 Database: Email 32021432-2d4e-4c54-b04c-f81b4120340c saved to database.

=== Workflow Output ===
Email sent: Hi Alex,

Thank you for summarizing the action items from this morning's meeting.
I have noted the three tasks and will begin working on them right away.
I'll aim to have the updated project timeline ready by Friday...
```

**Execution Flow:**
1. Email stored (285 chars > 100 threshold)
2. Analysis: NotSpam detected
3. **Multi-selection triggers BOTH branches:**
   - Branch 1: Draft reply (parallel)
   - Branch 2: Summarize email (parallel)
4. Summary merged into `AnalysisResult`
5. Database persists email WITH summary
6. Final output: Drafted reply

## Key Takeaways

### Multi-Selection Edge Groups
✅ **One-to-Many:** Single message can activate **multiple** downstream executors  
✅ **Parallel Execution:** Selected branches run concurrently  
✅ **Dynamic Selection:** Number of branches determined at runtime by `selection_func`  
✅ **Flexible Routing:** Return 0, 1, or N targets from selection function

### Selection Function Pattern
```python
def select_targets(message: T, target_ids: list[str]) -> list[str]:
    # Inspect message properties
    # Return list of target IDs to invoke
    
    if condition_a:
        return [target_ids[0]]  # Single branch
    elif condition_b:
        return [target_ids[1], target_ids[2]]  # PARALLEL branches
    else:
        return []  # No branches (workflow becomes idle)
```

### Parallel Branching Use Cases

**1. Parallel Processing:**
```python
# Long documents: summarize AND translate simultaneously
if len(doc) > threshold:
    return [summarize_id, translate_id]  # Both run in parallel
```

**2. Multi-Channel Notifications:**
```python
# High-priority alerts: send email AND SMS AND Slack
if priority == 'critical':
    return [email_id, sms_id, slack_id]  # All three
```

**3. Conditional Enrichment:**
```python
# Basic processing always, optional enrichment if needed
targets = [basic_process_id]
if needs_enrichment:
    targets.append(enrich_id)  # Add second branch
return targets
```

### Shared State for Coordination
**Why Share State?**
- Parallel branches need access to common data (email content)
- Avoid duplicating large objects in messages
- Coordinate results from multiple branches

**Pattern:**
```python
# Store once
await ctx.set_shared_state(f"email:{email_id}", email)

# Branch 1: Draft reply (retrieves email)
email = await ctx.get_shared_state(f"email:{email_id}")

# Branch 2: Summarize (retrieves same email)
email = await ctx.get_shared_state(f"email:{email_id}")
```

### Merging Parallel Results
**Problem:** Two parallel branches produce results; how to combine?

**Solution:** Merge executor updates shared state or sends enriched message
```python
@executor(id="merge_summary")
async def merge_summary(summary_response, ctx):
    # Get original analysis
    email_id = await ctx.get_shared_state("current_email_id")
    email = await ctx.get_shared_state(f"email:{email_id}")
    
    # Create enriched result with summary
    enriched = AnalysisResult(
        spam_decision="NotSpam",
        email_summary=summary.summary,  # From parallel branch!
        email_id=email_id,
        ...
    )
    await ctx.send_message(enriched)
```

### Conditional Database Persistence
**Two paths to database based on email length:**

**Short Emails:**
```python
# Direct edge with condition
.add_edge(to_analysis_result, database_access, 
          condition=lambda r: r.email_length <= THRESHOLD)
```

**Long Emails:**
```python
# Wait for summary branch to merge results
.add_edge(merge_summary, database_access)
```

**Why?**
- Short emails: No summary needed, persist immediately
- Long emails: Wait for summary to be generated and merged first

## Pattern Comparison

### Switch-Case (One-of-N)
```python
# Exactly ONE path executes
.add_switch_case_edge_group(
    classifier,
    [
        Case(condition=is_a, target=path_a),  # Exclusive
        Case(condition=is_b, target=path_b),  # Exclusive
        Default(target=path_c),               # Exclusive
    ],
)
```

### Multi-Selection (One-to-Many)
```python
# MULTIPLE paths can execute simultaneously
.add_multi_selection_edge_group(
    analyzer,
    [path_a, path_b, path_c],
    selection_func=lambda msg, ids: [ids[0], ids[1]]  # Parallel!
)
```

## Execution Flow Diagram

### Short NotSpam Email (≤100 chars):
```
store_email → analysis_agent → to_analysis_result
                                      │
                                      ├─→ submit_to_assistant → assistant → finalize
                                      │
                                      └─→ database_access
```
**Selection:** `[submit_to_email_assistant]` (1 branch)  
**Database:** Direct from `to_analysis_result`

### Long NotSpam Email (>100 chars):
```
store_email → analysis_agent → to_analysis_result
                                      │
                                      ├─→ submit_to_assistant → assistant → finalize
                                      │
                                      └─→ summarize_email → summary_agent → merge_summary → database_access
```
**Selection:** `[submit_to_email_assistant, summarize_email]` (2 branches!)  
**Database:** After summary merged

## Best Practices

### When to Use Multi-Selection
✅ **Parallel processing** of same data (summarize + translate)  
✅ **Multi-channel actions** (email + SMS + log)  
✅ **Conditional enrichment** (always process, sometimes enrich)  
✅ **Fan-out aggregation** (multiple analyzers, merge results)

### When NOT to Use Multi-Selection
❌ **Exclusive routing** (if/else) → Use `switch-case` or `edge conditions`  
❌ **Sequential processing** (A then B) → Use `add_edge(A, B)`  
❌ **Unrelated branches** → Create separate workflows

### Selection Function Guidelines
- **Keep logic simple** - complex decisions belong in executors
- **Return empty list** to short-circuit (workflow becomes idle)
- **Use descriptive IDs** in `target_ids` for readability
- **Consider order** if targets have dependencies (rare)

### Shared State Guidelines
- **Store large objects once** (emails, documents)
- **Pass IDs in messages** for lightweight coordination
- **Use namespacing** (prefixes) to avoid key collisions
- **Clean up** if state is very large (not shown in sample)

## Advanced Patterns

### Dynamic Fan-Out (Variable Branches)
```python
def select_targets(doc: Document, target_ids: list[str]) -> list[str]:
    # Route to ALL language-specific processors
    return [
        target_id for lang, target_id in zip(doc.languages, target_ids)
        if lang in doc.languages
    ]  # Could be 0, 1, or N branches!
```

### Priority-Based Selection
```python
def select_targets(alert: Alert, target_ids: list[str]) -> list[str]:
    email_id, sms_id, slack_id, pager_id = target_ids
    
    if alert.priority == 'critical':
        return [email_id, sms_id, slack_id, pager_id]  # All channels
    elif alert.priority == 'high':
        return [email_id, slack_id]  # Two channels
    else:
        return [email_id]  # Single channel
```

### Conditional Enrichment
```python
def select_targets(data: Data, target_ids: list[str]) -> list[str]:
    process_id, enrich_id, validate_id = target_ids
    
    # Always process
    targets = [process_id]
    
    # Conditionally add enrichment
    if data.needs_enrichment:
        targets.append(enrich_id)
    
    # Conditionally add validation
    if data.requires_validation:
        targets.append(validate_id)
    
    return targets  # 1, 2, or 3 branches
```

## Next Steps
- Compare with `switch_case_edge_group.ipynb` to see one-of-N vs one-to-many
- Explore `edge_condition.ipynb` for binary conditional routing
- Check `simple_loop.ipynb` for iterative patterns with feedback